In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# =============================================================================
# CAFA6 + ESM2 - PHIÊN BẢN TỐI ƯU TỐC ĐỘ (SPEED OPTIMIZED)
# =============================================================================

# 1. MOUNT DRIVE
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# 2. CONFIG & IMPORTS
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm

# --- CẤU HÌNH ĐƯỜNG DẪN ---
BASE_DIR = "/content/drive/MyDrive/CAFA6"

TRAIN_SEQ_PATH   = os.path.join(BASE_DIR, "Train/train_sequences.fasta")
TRAIN_TERMS_PATH = os.path.join(BASE_DIR, "Train/train_terms.tsv")
TEST_SEQ_PATH    = os.path.join(BASE_DIR, "Test/testsuperset.fasta")
SAMPLE_SUB_PATH  = os.path.join(BASE_DIR, "sample_submission.tsv")

EMBED_TRAIN_PATH = os.path.join(BASE_DIR, "train_embeddings_esm2_t6.npy")
EMBED_TEST_PATH  = os.path.join(BASE_DIR, "test_embeddings_esm2_t6.npy")

# --- [TỐI ƯU]  ---
EMBED_BATCH_SIZE = 64
BATCH_SIZE_TRAIN = 256
HIDDEN_DIM = 512
LR = 1e-3
EPOCHS = 10
MIN_GO_FREQ = 50

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

# Cài thư viện ESM
try:
    import esm
except ImportError:
    import esm

# 3. HÀM ĐỌC FASTA
def read_fasta(path):
    ids, seqs = [], []
    with open(path, "r") as f:
        current_id = None
        current_seq = []
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                if current_id is not None:
                    ids.append(current_id)
                    seqs.append("".join(current_seq))

                raw = line[1:].split()[0]
                if "|" in raw:
                    parts = raw.split("|")
                    if len(parts) >= 2 and parts[0] in ("sp", "tr"):
                        current_id = parts[1]
                    else:
                        current_id = raw
                else:
                    current_id = raw

                current_seq = []
            else:
                current_seq.append(line)

        if current_id:
            ids.append(current_id)
            seqs.append("".join(current_seq))

    return pd.DataFrame({"EntryID": ids, "sequence": seqs})

# 4. LOAD MODEL ESM2
print("Loading ESM2 Model...")
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
model = model.to(device)
model.eval()
batch_converter = alphabet.get_batch_converter()
repr_layer = 6
embed_dim = 320

# 5. HÀM TẠO EMBEDDING
def get_memmap_embedding(ids, seqs, save_path, batch_size=64):
    n_samples = len(seqs)

    # Nếu file embedding đã tồn tại thì load lại
    if os.path.exists(save_path):
        print(f"✅ Đã tìm thấy file: {save_path}")
        print("   -> Đang load lại từ ổ cứng (BỎ QUA chạy lại ESM2)...")
        try:
            mmap = np.memmap(save_path, dtype='float16', mode='r',
                             shape=(n_samples, embed_dim))
            return mmap
        except Exception as e:
            print(f"   -> File lỗi, sẽ chạy lại. Lỗi: {e}")

    print(f"🚀 Không thấy file cũ, bắt đầu chạy ESM2 (Sẽ mất thời gian lần đầu)...")
    print(f"   -> Lưu tại: {save_path}")

    mmap_arr = np.memmap(save_path, dtype='float16', mode='w+',
                         shape=(n_samples, embed_dim))

    for i in tqdm(range(0, n_samples, batch_size), desc="Embedding"):
        batch_ids = ids[i : i + batch_size]
        batch_seqs_raw = seqs[i : i + batch_size]
        # Cắt chuỗi dài quá giới hạn
        batch_seqs = [s[:1022] for s in batch_seqs_raw]

        data = list(zip(batch_ids, batch_seqs))
        labels, strs, toks = batch_converter(data)
        toks = toks.to(device)

        with torch.no_grad():
            # Mixed precision để nhanh hơn
            with torch.cuda.amp.autocast():
                results = model(toks, repr_layers=[repr_layer],
                                return_contacts=False)
                token_reps = results["representations"][repr_layer]
                # Average pooling trên chiều sequence (bỏ [CLS], [EOS])
                emb = token_reps[:, 1:-1].mean(1)

            emb_cpu = emb.float().cpu().numpy().astype(np.float16)

        current_len = len(batch_seqs)
        mmap_arr[i : i + current_len] = emb_cpu

        del toks, results, token_reps, emb, emb_cpu
        if i % (batch_size * 10) == 0:
            torch.cuda.empty_cache()

    mmap_arr.flush()
    return np.memmap(save_path, dtype='float16', mode='r',
                     shape=(n_samples, embed_dim))

# 6. XỬ LÝ TRAIN
print("\n--- 1. PREPARING TRAIN DATA ---")
train_seq_df = read_fasta(TRAIN_SEQ_PATH)
train_terms = pd.read_csv(TRAIN_TERMS_PATH, sep="\t")

go_counts = train_terms["term"].value_counts()
valid_go = set(go_counts[go_counts >= MIN_GO_FREQ].index)
train_terms_filt = train_terms[train_terms["term"].isin(valid_go)].copy()

entry_to_terms = (
    train_terms_filt.groupby("EntryID")["term"]
    .apply(list)
    .reset_index()
    .rename(columns={"term": "terms"})
)

train_df = train_seq_df.merge(entry_to_terms, on="EntryID", how="inner")
print(f"Số lượng mẫu train: {len(train_df)}")

# Chạy Embedding Train
X_train_mmap = get_memmap_embedding(
    train_df["EntryID"].tolist(),
    train_df["sequence"].tolist(),
    EMBED_TRAIN_PATH,
    batch_size=EMBED_BATCH_SIZE,
)

print("Encoding Labels...")
mlb = MultiLabelBinarizer(sparse_output=True)
Y_sparse = mlb.fit_transform(train_df["terms"])
Y_dense = Y_sparse.toarray().astype(np.float32)
n_labels = len(mlb.classes_)

idx_train, idx_val = train_test_split(
    np.arange(len(train_df)),
    test_size=0.1,
    random_state=42
)

del train_seq_df, train_terms, train_terms_filt, entry_to_terms, Y_sparse
gc.collect()

# 7. MODEL & DATASET
class MemmapDataset(Dataset):
    def __init__(self, memmap_array, labels, indices):
        self.memmap_array = memmap_array
        self.labels = labels
        self.indices = indices

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        real_idx = self.indices[idx]
        return (
            torch.tensor(self.memmap_array[real_idx], dtype=torch.float32),
            torch.tensor(self.labels[real_idx], dtype=torch.float32),
        )

class MLP(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(hidden_dim, out_dim),
        )

    def forward(self, x):
        return self.net(x)

# 8. TRAINING
print("\n--- 2. TRAINING MLP ---")
train_ds = MemmapDataset(X_train_mmap, Y_dense, idx_train)
val_ds   = MemmapDataset(X_train_mmap, Y_dense, idx_val)

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE_TRAIN,
    shuffle=True,
    pin_memory=True,
    num_workers=2,
)
val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE_TRAIN,
    shuffle=False,
    num_workers=2,
)

clf = MLP(embed_dim, HIDDEN_DIM, n_labels).to(device)
optimizer = torch.optim.Adam(clf.parameters(), lr=LR)
criterion = nn.BCEWithLogitsLoss()

best_f1 = 0.0

for epoch in range(EPOCHS):
    clf.train()
    total_loss = 0.0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = clf(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    clf.eval()
    preds_list, targets_list = [], []
    with torch.no_grad():
        for x, y in val_loader:
            x = x.to(device)
            p = torch.sigmoid(clf(x))
            preds_list.append((p > 0.3).float().cpu().numpy())
            targets_list.append(y.numpy())

    f1 = f1_score(
        np.vstack(targets_list),
        np.vstack(preds_list),
        average="micro"
    )
    print(
        f"Epoch {epoch+1} | "
        f"Loss: {total_loss/len(train_loader):.4f} | "
        f"Val F1: {f1:.4f}"
    )

    if f1 > best_f1:
        best_f1 = f1
        torch.save(clf.state_dict(), "best_model.pth")

print(f"Best Val F1: {best_f1:.4f}")

del train_ds, val_ds, train_loader, val_loader, Y_dense
gc.collect()

# 9. PREDICT TEST
print("\n--- 3. PREDICTING TEST DATA ---")
clf.load_state_dict(torch.load("best_model.pth"))
clf.eval()

test_df = read_fasta(TEST_SEQ_PATH)

# Chạy Embedding Test
X_test_mmap = get_memmap_embedding(
    test_df["EntryID"].tolist(),
    test_df["sequence"].tolist(),
    EMBED_TEST_PATH,
    batch_size=EMBED_BATCH_SIZE,
)

class TestDataset(Dataset):
    def __init__(self, memmap_array):
        self.memmap_array = memmap_array

    def __len__(self):
        return len(self.memmap_array)

    def __getitem__(self, idx):
        return torch.tensor(self.memmap_array[idx], dtype=torch.float32)

test_loader = DataLoader(
    TestDataset(X_test_mmap),
    batch_size=BATCH_SIZE_TRAIN * 2,
    shuffle=False,
    num_workers=2,
)

all_probs = []
with torch.no_grad():
    for x in tqdm(test_loader, desc="Predicting"):
        all_probs.append(
            torch.sigmoid(clf(x.to(device))).cpu().numpy()
        )

final_probs = np.vstack(all_probs)

# 10. SUBMISSION
print("\n--- 4. CREATING SUBMISSION (OPTIMIZED) ---")

print("Building prediction DataFrame...")
test_ids = test_df["EntryID"].tolist()
pred_rows = []

# Lấy Top 50 điểm cao nhất cho mỗi protein để giảm dung lượng
top_k = 50

for idx, entry_id in enumerate(tqdm(test_ids, desc="Filtering Top Scores")):
    scores = final_probs[idx]

    # Lấy index của top K điểm cao nhất
    if len(scores) > top_k:
        top_indices = np.argpartition(scores, -top_k)[-top_k:]
    else:
        top_indices = np.arange(len(scores))

    for term_idx in top_indices:
        score = scores[term_idx]
        if score > 0.001:
            pred_rows.append(
                {
                    "EntryID": entry_id,
                    "term": mlb.classes_[term_idx],
                    "score": float(score),
                }
            )

my_preds_df = pd.DataFrame(pred_rows)

# Bước 2: Load Sample Submission
if os.path.exists(SAMPLE_SUB_PATH):
    print(f"Found sample submission at: {SAMPLE_SUB_PATH}")
    try:
        # Đọc thử 1 dòng để check cột trước khi dùng usecols
        header_check = pd.read_csv(SAMPLE_SUB_PATH, sep="\t", nrows=1)
        print(f"   -> Columns detected: {header_check.columns.tolist()}")

        if "EntryID" in header_check.columns and "term" in header_check.columns:
            print("   -> Template format looks correct. merging...")
            sample_df = pd.read_csv(
                SAMPLE_SUB_PATH,
                sep="\t",
                usecols=["EntryID", "term"],
            )

            # Ghép điểm dự đoán vào bảng mẫu (Left Join)
            submission = sample_df.merge(
                my_preds_df,
                on=["EntryID", "term"],
                how="left",
            )
            submission["score"] = submission["score"].fillna(0.001)
        else:
            print("   ⚠️ WARNING: Sample file columns do not match ['EntryID', 'term']. Skipping merge.")
            print("   -> Using raw predictions instead.")
            submission = my_preds_df
    except Exception as e:
        print(f"   ⚠️ ERROR reading sample submission: {e}")
        print("   -> Using raw predictions instead.")
        submission = my_preds_df
else:
    print("Sample submission not found. Exporting all valid predictions...")
    submission = my_preds_df

# Format lại điểm số
submission["score"] = submission["score"].map(lambda x: f"{x:.3f}")

output_path = "/content/submission_esm2_optimized.tsv"
submission.to_csv(output_path, sep="\t", index=False)
print(f"✅ Xong! File lưu tại: {output_path}")

try:
    from google.colab import files
    files.download(output_path)
except Exception:
    pass
